In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../menu.csv')

In [3]:
len(df)

50

In [4]:
df.head()

,name,price,energy,protein,fat,carbohydrates,salt,calcium,vegetable,img_url
0,ローストンカツおろしソース,264.0,406.0,15.3,26.0,25.3,2.4,16.0,16.0,https://west2-univ.jp/menu_img/png_sp/824022.png
1,チキンカツ柚子胡椒マヨ,220.0,399.0,16.4,27.9,18.1,1.2,11.0,0.0,https://west2-univ.jp/menu_img/png_sp/824082.png
2,ハンバーグトマトソース,220.0,221.0,15.0,11.2,13.8,1.6,38.0,37.0,https://west2-univ.jp/menu_img/png_sp/824037.png
3,豚ブロッコリー和風炒め,264.0,280.0,18.1,18.6,9.6,1.7,85.0,200.0,https://west2-univ.jp/menu_img/png_sp/814202.png
4,ビーフシチュー,264.0,316.0,9.2,16.2,30.6,1.7,22.0,80.0,https://west2-univ.jp/menu_img/png_sp/814126.png


In [5]:
#　必要量を入れた辞書
r_dic = {
    'energy': 2650 / 3,
    'protein': 65 / 3 ,
    'fat': 60 / 3,
    'carbohydrates': 300 / 3,
    'salt' : 7.5 / 3, #以下
    'calcium' : 800 / 3,
    'vegetable': 350 / 3 # 以上    
}

決定変数  
$x_{s}$ : メニューsを選ぶかどうか

In [6]:
import pulp 

In [7]:
# df['name'].to_dict()

In [8]:
S = list(range(len(df)))

In [9]:
# 上限金額
max_price = 1000
# 食べたい商品
T_plus = []
T_minus = []

In [10]:
# 問題設定
problem = pulp.LpProblem('Problem Name', pulp.LpMinimize) 
xs = pulp.LpVariable.dicts('x', S, 0, 1, 'Integer')
# スラック変数の定義
nutrition_list = ['energy', 'protein', 'fat', 'carbohydrates', 'salt', 'calcium', 'vegetable']
zs = pulp.LpVariable.dict('z', nutrition_list, lowBound=0,cat='Continuous')

/usr/local/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [11]:
zs

{'energy': z_energy,
 'protein': z_protein,
 'fat': z_fat,
 'carbohydrates': z_carbohydrates,
 'salt': z_salt,
 'calcium': z_calcium,
 'vegetable': z_vegetable}

In [12]:
# 目的関数　slack変数z
problem += pulp.lpSum(zs)

# 制約式
# 好き嫌い制約
for idx in T_plus:
    problem += xs[idx] == 1
for idx in T_minus:
    problem += xs[idx] == 0

# 金額制約
problem += pulp.lpSum(df['price'][i]*xs[s] for i,s in enumerate(S)) <= max_price

# 栄養制約
for nut in nutrition_list:    
    if nut == 'salt':
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] <= zs[nut]*r_dic[nut]
    if nut == 'vegetable':
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] >= -zs[nut]*r_dic[nut]
    else :
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] <= zs[nut]*r_dic[nut]
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] >= -zs[nut]*r_dic[nut]

In [13]:
sol = problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/kc/5vks2chj5jg_zq9fvglb01jr0000gp/T/cd0cb480103240629b4c671e287083ce-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/kc/5vks2chj5jg_zq9fvglb01jr0000gp/T/cd0cb480103240629b4c671e287083ce-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 862 RHS
At line 878 BOUNDS
At line 929 ENDATA
Problem MODEL has 15 rows, 57 columns and 734 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.218619 - 0.00 seconds
Cgl0004I processed model has 14 rows, 57 columns (50 integer (50 of which binary)) and 687 elements
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.97544
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 0.978974 iterations 15
Cbc0038I Solution found of 0.978974
Cbc003

In [14]:
pulp.LpStatus[sol]

'Optimal'

In [15]:
ret_dic = {}
total_price = 0
for nut in nutrition_list:
    ret_dic[nut] = 0.
for i in xs:
    if xs[i].value():
        total_price += df['price'][i]
        for nut in nutrition_list:
            ret_dic[nut] += df[nut][i]

In [16]:
for nut in nutrition_list:
    print(nut, ret_dic[nut] - r_dic[nut])

energy -118.33333333333337
protein 1.4333333333333336
fat -0.3000000000000007
carbohydrates 19.400000000000006
salt -0.09999999999999964
calcium 17.333333333333314
vegetable 4.333333333333329


In [17]:
for i in zs:
    print(zs[i].value())

0.13396226
0.066153846
0.015
0.194
0.04
0.065
0.0


In [18]:
print(f"目的関数 : {problem.objective.value()}")
print(f"価格 : {total_price}　円")
for i in range(len(S)):
    if xs[i].value():
        print(df['name'][i], df['price'][i])

目的関数 : 0.514116106
価格 : 555.0　円
ししゃもフライ 176.0
ちくわの磯部揚げ 110.0
ブロッコリーピーナッツ和え 88.0
スライスオクラ 66.0
ライス 115.0


In [19]:
df.iloc[[i for i,v in enumerate(xs.values()) if v.value() == 1]]

,name,price,energy,protein,fat,carbohydrates,salt,calcium,vegetable,img_url
6,ししゃもフライ,176.0,135.0,6.2,8.5,8.5,1.1,176.0,0.0,https://west2-univ.jp/menu_img/png_sp/814295.png
9,ちくわの磯部揚げ,110.0,162.0,4.9,9.8,13.5,0.8,40.0,0.0,https://west2-univ.jp/menu_img/png_sp/814465.png
14,ブロッコリーピーナッツ和え,88.0,45.0,3.8,1.4,4.6,0.5,23.0,71.0,https://west2-univ.jp/menu_img/png_sp/814413.png
18,スライスオクラ,66.0,15.0,1.0,0.0,4.0,0.0,45.0,50.0,https://west2-univ.jp/menu_img/png_sp/814454.png
35,ライス,115.0,408.0,7.2,0.0,88.8,0.0,0.0,0.0,https://west2-univ.jp/menu_img/png_sp/814702.png
